In [10]:
import pandas as pd
import numpy as np
import re  
# "\n\ns.".replace("\n","").strip('.')
path = "/Users/luokui/Documents/Akeras/yelp_2013.csv"
data = pd.read_csv(path)

In [3]:
def get_data(path):
    token = {}
    data = pd.read_csv(path)
    last_data = {}
    last_data.setdefault("data", [])
    last_data.setdefault("label", [])
    
    for line in data.text:
        line = line.strip().split(" ")
        temp_line = [word.replace('\n', '').strip('[.,""]').lower() for word in line if len(word)>0]
        last_data["data"].append(temp_line)
        for word in temp_line:
            token.setdefault(word,0)
            token[word] += 1
    for line in data.stars:
        last_data["label"].append(line)
    
    word2indx = {}
    word_list = []
    for key in token.keys():
        word_list.append(key)
        word2indx.setdefault(key, len(word_list)-1)
    return token, word2indx, word_list, last_data
        
tokens, word2indx, word_list, last_data= get_data(path)

In [4]:
"""
token = {}
for line in data.text[:10]:
    temp_line = line.strip().split(" ")
    for word in temp_line:
        word = word.replace('\n', '').strip('[.,""]')
        token.setdefault(word, 0)
        token[word] += 1
"""

In [5]:
# word_list
da = []
for line in last_data['data']:
    temp = []
    for key in line:
        index = word2indx[key]
        temp.append(index)
    da.append(temp)

In [ ]:
"""
def get_batch(das, labels, batch_size=128):
    m = len(das)
    index = np.random.permutation(m)
    das = np.array(das)[index]
    labels = np.array(labels)[index]
    n_batch = int(m / batch_size)
    for i in range(n_batch):
        yield das[i * batch_size : (i + 1) * batch_size], labels[i * batch_size : (i+1) * batch_size]
"""

In [23]:
import keras
from keras.layers import Embedding, Input, GRU, Wrapper, Bidirectional, Dense, Lambda
from keras import Sequential
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.utils import  to_categorical
import keras.backend as K

In [8]:
vocab_size = len(word2indx)
input_len = 72
print(vocab_size, input_len)
atten_size  = 10
hidden_size = 50
hop = 3

687583 72


In [11]:
# 切分数据
data  = pad_sequences(da, maxlen=input_len, padding="post", truncating="post")
labels = to_categorical(np.array(last_data['label']) - 1)

def split_data(data, label):
    m = len(data)
    data = np.array(data)
    label = np.array(label)
    train_data = data[:int(m * 0.7)]
    train_label = label[:int(m * 0.7)]
    valid_data = data[int(m * 0.7) : int(m * 0.9)]
    valid_label = label[int(m * 0.7) : int(m * 0.9)]
    test_data = data[int(m * 0.9) :]
    test_label = label[int(m * 0.9) : ]
    return train_data, train_label, valid_data, valid_label, test_data, test_label
    
train_data, train_label, valid_data, valid_label, test_data, test_label = split_data(data, labels)  

In [28]:
def change_shape(tensors):
    atten_3 = K.permute_dimensions(tensors, [0, 2, 1])
    # atten_3 = K.reshape(tensors, [-1, 3, 72])                           # 这种搞法有问题的
    atten_result = K.batch_dot(atten_3, rnn_result)
    flatted = K.batch_flatten(atten_result)                               # 注意这里的两个步骤
    atten_flatten = K.reshape(flatted, [-1, hop * 2 * hidden_size])
    return atten_flatten


# 对 attention norm
def norm_atten(tensors):
    atten2_t = K.batch_dot(K.permute_dimensions(tensors, [0, 2, 1]), tensors) - 1
    atten2_t = K.pow(atten2_t, 2)
    atten2_norm = K.sqrt(K.mean(K.sum(K.sum(atten2_t, axis=-1), axis=-1)))
    return atten2_norm

In [29]:
inputs = Input(shape=(input_len,))
embed = Embedding(input_dim=vocab_size + 1, input_length=input_len, output_dim=100)
embed_inputs = embed(inputs)

rnn_model = Bidirectional(GRU(hidden_size, return_sequences=True))
rnn_result = rnn_model(embed_inputs)                                       # b, n , zu
atten_1 = Dense(atten_size, activation="tanh", use_bias=False)(rnn_result) # b * n * da
atten_2 = Dense(hop, activation="softmax", use_bias=False)(atten_1)        # b * n * r

atten_flatten = Lambda(change_shape)(atten_2)             # b * r * n   // b * n * 2u
atten2_norms = Lambda(norm_atten)(atten_2)

In [18]:
# y_class = 5
# fc = 10
fc_model = Dense(10)(atten_flatten)
out_put = Dense(5, activation="softmax")(fc_model)
model = Model(inputs, out_put)
model.summary()

TensorShape([Dimension(None), Dimension(5)])

In [24]:
# 可自定义 损失函数，及使用 r * r 的正则化项目， 在其他电脑。 sliced rnn
# model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])
def my_loss(y, y_pred):
    return K.mean(K.categorical_crossentropy(y, y_pred, from_logits=True)) + atten2_norms

model.compile(optimizer="adam", loss=my_loss, metrics=["acc"])

In [27]:
# just test !!
for iters in range(4):
    print("iter is {}".format(iters))
    """
    model.fit(train_data[:10000], train_label[:10000], verbose=1, batch_size=512*4, epochs=10,
             validation_data=(valid_data[:1000], valid_label[:1000]))
    if iters % 2 ==0:
        score = model.evaluate(test_data, test_label)
    """

iter is 0
iter is 1
iter is 2
iter is 3
